In [2]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

2024-11-27 19:21:38.123812: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import gensim.downloader as api

In [4]:
from logic import processing

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Flotchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Flotchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
data = processing.load_data('Combined Data.csv')

In [6]:
df = processing.preproc(data,bi = True)

In [7]:
df.head()

,Unnamed: 0,statement,status,clean,label
0,0,oh my gosh,Anxiety,oh gosh,0
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleep confuse mind restless heart tune,0
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,wrong back dear forward doubt stay restless re...,0
3,3,I've shifted my focus to something else but I'...,Anxiety,ive shift focus something else im still worry,0
4,4,"I'm restless and restless, it's been a month n...",Anxiety,im restless restless month boy mean,0


In [8]:
X = df.clean
y= df.label

Either train a model --> Word2vec()
Either load a pre-trained model --W api.load(model)

In [5]:
#Choose a model embedding
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [6]:
word2vec_transfer = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [7]:
#Parameters
vector_size=60
min_count=10
window=10
maxlen=200

In [9]:
def embed_sentence(word2vec, sentence):
    """Embed a sentence by averaging the embeddings of its words."""
    embedded_words = [word2vec.wv[word] for word in sentence if word in word2vec.wv]
    if not embedded_words:
        return np.zeros(word2vec.vector_size)  # Phrase sans mots connus -> vecteur nul
    return np.mean(embedded_words, axis=0)

def embedding(word2vec, sentences):
    """Embed a list of sentences."""
    return np.array([embed_sentence(word2vec, sentence.split()) for sentence in sentences])


In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

In [13]:
# Entraîner Word2Vec
word2vec = Word2Vec(sentences=[sentence.split() for sentence in X], 
                    vector_size=60, 
                    min_count=5, 
                    window=5)

# Embedding des données
X_embed = embedding(word2vec, X)

# Modèle Naïve Bayes
model = LogisticRegression(max_iter=1000)

# Cross-validation
cv_results = cross_validate(
    model,
    X_embed,
    y,
    scoring=['accuracy', 'precision_macro', 'recall_macro'],
    cv=5,  # Par exemple, 5-fold cross-validation
)

print(cv_results)


{'fit_time': array([0.26083589, 0.39997792, 0.2699008 , 0.30358815, 0.35250974]), 'score_time': array([0.01820731, 0.01452804, 0.01770592, 0.01406384, 0.01313019]), 'test_accuracy': array([0.8712414 , 0.87274955, 0.84400038, 0.84040347, 0.8280543 ]), 'test_precision_macro': array([0.85376249, 0.87621042, 0.85012056, 0.81258978, 0.80269558]), 'test_recall_macro': array([0.83945383, 0.81850213, 0.7734519 , 0.81411566, 0.78450644])}


In [14]:
cv_results['test_accuracy'].mean()

0.8512898191680218